## Exercise: Classifying the MNIST Dataset with Keras
---

In [1]:
import tensorflow as tf 
from tensorflow.python.client import device_lib
print("Is GPU available? {0}".format(tf.test.is_gpu_available()))

local_device_protos = device_lib.list_local_devices()
print("List of GPUs: ", [x.name for x in local_device_protos if x.device_type == 'GPU'])

print("List of devices: ")
print(device_lib.list_local_devices())

Is GPU available? False
List of GPUs:  []
List of devices: 
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9597230160833706048
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 1060424767575803676
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 9810048291066593611
physical_device_desc: "device: XLA_GPU device"
]


In [2]:
import tensorflow as tf
# Load MNIST dataset
# 60000 handwritten drawings of digits, uploaded as 28x28 greyscale images 
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Describe the data
img_rows = 28
img_cols = 28
num_classes = 10  # classifying numbers between 0-9

### Data Pipeline
---
Data is modified to a format that can be used to train the model or be evaluated by the model
* Reshape
* Normalize

In [3]:
# Tensorflow by default supports the ndarray format: 
#    NHWC (Number of samples, Height of each sample, Width of each sample, # of channels)
# Need to convert input data from NHW to NHWC, where C=1 because images are greyscale
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

# Define input shape of a single record
input_shape = (img_rows, img_cols, 1)

# Normalize input data
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert labels from multi-class values to one-hot encoded vectors
#  3 => 0 0 0 1 0 0 0 0 0 0 and 1 => 0 1 0 0 0 0 0 0 0 0 
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

### Modeling
___

### Model 1

In [6]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dropout, Softmax
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential()
model.add(layer=Conv2D(64, kernel_size=(3,3),
                       activation='relu',
                       input_shape=input_shape))
print(model.output_shape)
model.add(layer=Conv2D(32, kernel_size=(3,3),
                       activation='relu'))
print(model.output_shape)
model.add(MaxPooling2D(pool_size=(2,2)))
print(model.output_shape)
model.add(Flatten())
print(model.output_shape)
model.add(Dropout(0.5))
print(model.output_shape)
model.add(Dense(num_classes, activation='softmax'))
print(model.output_shape)

#   define compile to minimize categorical loss, use ada delta optimized, and optimize to maximizing accuracy
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

#   Train the model and test/validate the mode with the test data after each cycle (epoch) through the training data
#   Return history of loss and accuracy for each epoch
batch_size = 20
epochs = 20
cbk_early_stopping = EarlyStopping(monitor='val_acc', patience=2, mode='max')

hist = model.fit(x_train, y_train,
                 batch_size=batch_size,
                 epochs=epochs,
                 verbose=1,
                 validation_data=(x_test, y_test), 
                 callbacks=[cbk_early_stopping])

#   Evaluate the model with the test data to get the scores on "real" data.
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

(None, 26, 26, 64)
(None, 24, 24, 32)
(None, 12, 12, 32)
(None, 4608)
(None, 4608)
(None, 10)
Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 13s 222us/step - loss: 0.1702 - acc: 0.9473 - val_loss: 0.0526 - val_acc: 0.9832
Epoch 2/20
60000/60000 [==============================] - 13s 217us/step - loss: 0.0759 - acc: 0.9770 - val_loss: 0.0501 - val_acc: 0.9838
Epoch 3/20
60000/60000 [==============================] - 13s 217us/step - loss: 0.0590 - acc: 0.9816 - val_loss: 0.0383 - val_acc: 0.9868
Epoch 4/20
60000/60000 [==============================] - 13s 212us/step - loss: 0.0498 - acc: 0.9842 - val_loss: 0.0361 - val_acc: 0.9883
Epoch 5/20
60000/60000 [==============================] - 13s 209us/step - loss: 0.0455 - acc: 0.9854 - val_loss: 0.0361 - val_acc: 0.9884
Epoch 6/20
60000/60000 [==============================] - 13s 210us/step - loss: 0.0409 - acc: 0.9873 - val_loss: 0.0339 - val_acc: 0.9886
Epoch 7/20
60000/60000

### Model 2

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dropout, Softmax
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential()
model.add(layer=Conv2D(32, kernel_size=(3,3),
                       activation='relu',
                       input_shape=input_shape))
print(model.output_shape)
model.add(Flatten())
print(model.output_shape)
model.add(Dense(num_classes, activation='softmax'))
print(model.output_shape)

#   define compile to minimize categorical loss, use ada delta optimized, and optimize to maximizing accuracy
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

#   Train the model and test/validate the mode with the test data after each cycle (epoch) through the training data
#   Return history of loss and accuracy for each epoch
batch_size = 20
epochs = 20
cbk_early_stopping = EarlyStopping(monitor='val_acc', patience=2, mode='max')

hist = model.fit(x_train, y_train,
                 batch_size=batch_size,
                 epochs=epochs,
                 verbose=1,
                 validation_data=(x_test, y_test), 
                 callbacks=[cbk_early_stopping])

#   Evaluate the model with the test data to get the scores on "real" data.
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

(None, 26, 26, 32)
(None, 21632)
(None, 10)
Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 10s 173us/step - loss: 0.1615 - acc: 0.9519 - val_loss: 0.0804 - val_acc: 0.9737
Epoch 2/20
60000/60000 [==============================] - 10s 169us/step - loss: 0.0637 - acc: 0.9811 - val_loss: 0.0644 - val_acc: 0.9789
Epoch 3/20
60000/60000 [==============================] - 10s 169us/step - loss: 0.0448 - acc: 0.9861 - val_loss: 0.0604 - val_acc: 0.9809
Epoch 4/20
60000/60000 [==============================] - 10s 172us/step - loss: 0.0324 - acc: 0.9902 - val_loss: 0.0593 - val_acc: 0.9818
Epoch 5/20
60000/60000 [==============================] - 10s 172us/step - loss: 0.0237 - acc: 0.9925 - val_loss: 0.0637 - val_acc: 0.9815
Epoch 6/20
60000/60000 [==============================] - 10s 173us/step - loss: 0.0173 - acc: 0.9942 - val_loss: 0.0745 - val_acc: 0.9805
Test loss: 0.07452482019617164
Test accuracy: 0.9805


## Model 3

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dropout, Softmax
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential()
model.add(layer=Conv2D(64, kernel_size=(3,3),
                       activation='relu',
                       input_shape=input_shape))
model.add(layer=Conv2D(32, kernel_size=(3,3),
                       activation='relu'))
print(model.output_shape)
model.add(Flatten())
print(model.output_shape)
model.add(Dense(num_classes, activation='softmax'))
print(model.output_shape)

#   define compile to minimize categorical loss, use ada delta optimized, and optimize to maximizing accuracy
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

#   Train the model and test/validate the mode with the test data after each cycle (epoch) through the training data
#   Return history of loss and accuracy for each epoch
batch_size = 20
epochs = 20
cbk_early_stopping = EarlyStopping(monitor='val_acc', patience=2, mode='max')

hist = model.fit(x_train, y_train,
                 batch_size=batch_size,
                 epochs=epochs,
                 verbose=1,
                 validation_data=(x_test, y_test), 
                 callbacks=[cbk_early_stopping])

#   Evaluate the model with the test data to get the scores on "real" data.
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

(None, 24, 24, 32)
(None, 18432)
(None, 10)
Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 13s 223us/step - loss: 0.1146 - acc: 0.9663 - val_loss: 0.0714 - val_acc: 0.9780
Epoch 2/20
60000/60000 [==============================] - 12s 201us/step - loss: 0.0477 - acc: 0.9857 - val_loss: 0.0502 - val_acc: 0.9839
Epoch 3/20
60000/60000 [==============================] - 12s 203us/step - loss: 0.0302 - acc: 0.9907 - val_loss: 0.0575 - val_acc: 0.9837
Epoch 4/20
60000/60000 [==============================] - 12s 206us/step - loss: 0.0197 - acc: 0.9934 - val_loss: 0.0627 - val_acc: 0.9826
Test loss: 0.06268864197495859
Test accuracy: 0.9826


In [5]:
model.metrics_names

['loss', 'acc']

In [5]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

local_device_protos = device_lib.list_local_devices()
print([x.name for x in local_device_protos if x.device_type == 'GPU'])

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8942342153330756977
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 11493605629626433446
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 1840248520100869937
physical_device_desc: "device: XLA_GPU device"
]
[]


## Speed Test
---

In [6]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dropout, Softmax
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential()
model.add(layer=Conv2D(32, kernel_size=(3,3),
                       activation='relu',
                       input_shape=input_shape))
print(model.output_shape)
model.add(layer=Conv2D(32, kernel_size=(3,3),
                       activation='relu'))
print(model.output_shape)
model.add(Flatten())
print(model.output_shape)
model.add(Dropout(0.5))
print(model.output_shape)
model.add(Dense(num_classes, activation='softmax'))
print(model.output_shape)

#   define compile to minimize categorical loss, use ada delta optimized, and optimize to maximizing accuracy
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

#   Train the model and test/validate the mode with the test data after each cycle (epoch) through the training data
#   Return history of loss and accuracy for each epoch
batch_size = 20
epochs = 20
cbk_early_stopping = EarlyStopping(monitor='val_acc', patience=2, mode='max')

hist = model.fit(x_train, y_train,
                 batch_size=batch_size,
                 epochs=epochs,
                 verbose=1,
                 validation_data=(x_test, y_test), 
                 callbacks=[cbk_early_stopping])

#   Evaluate the model with the test data to get the scores on "real" data.
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

#   Train the model and test/validate the mode with the test data after each cycle (epoch) through the training data
#   Return history of loss and accuracy for each epoch
batch_size = 20
epochs = 20
cbk_early_stopping = EarlyStopping(monitor='val_acc', patience=2, mode='max')

hist = model.fit(x_train, y_train,
                 batch_size=batch_size,
                 epochs=epochs,
                 verbose=1,
                 validation_data=(x_test, y_test), 
                 callbacks=[cbk_early_stopping])


Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 64s 1ms/step - loss: 0.6987 - acc: 0.7820 - val_loss: 0.3171 - val_acc: 0.9099
Epoch 2/20
60000/60000 [==============================] - 64s 1ms/step - loss: 0.3791 - acc: 0.8873 - val_loss: 0.2797 - val_acc: 0.9205
Epoch 3/20
60000/60000 [==============================] - 66s 1ms/step - loss: 0.3307 - acc: 0.9008 - val_loss: 0.2389 - val_acc: 0.9335
Epoch 4/20
60000/60000 [==============================] - 72s 1ms/step - loss: 0.2989 - acc: 0.9104 - val_loss: 0.2039 - val_acc: 0.9451
Epoch 5/20
60000/60000 [==============================] - 74s 1ms/step - loss: 0.2618 - acc: 0.9224 - val_loss: 0.1689 - val_acc: 0.9532
Epoch 6/20
60000/60000 [==============================] - 75s 1ms/step - loss: 0.2246 - acc: 0.9327 - val_loss: 0.1375 - val_acc: 0.9597
Epoch 7/20
60000/60000 [==============================] - 71s 1ms/step - loss: 0.1945 - acc: 0.9413 - val_loss: 0.1165 - val_acc

In [7]:
from tensorflow.keras.callbacks import EarlyStopping

#   Train the model and test/validate the mode with the test data after each cycle (epoch) through the training data
#   Return history of loss and accuracy for each epoch
batch_size = 20
epochs = 20
cbk_early_stopping = EarlyStopping(monitor='val_acc', patience=2, mode='max')

hist = model.fit(x_train, y_train,
                 batch_size=batch_size,
                 epochs=epochs,
                 verbose=1,
                 validation_data=(x_test, y_test), 
                 callbacks=[cbk_early_stopping])


Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 13s 213us/step - loss: 0.1733 - acc: 0.9477 - val_loss: 0.0530 - val_acc: 0.9830
Epoch 2/20
60000/60000 [==============================] - 12s 195us/step - loss: 0.0760 - acc: 0.9769 - val_loss: 0.0469 - val_acc: 0.9852
Epoch 3/20
60000/60000 [==============================] - 12s 194us/step - loss: 0.0593 - acc: 0.9821 - val_loss: 0.0379 - val_acc: 0.9877
Epoch 4/20
60000/60000 [==============================] - 12s 194us/step - loss: 0.0497 - acc: 0.9846 - val_loss: 0.0388 - val_acc: 0.9871
Epoch 5/20
60000/60000 [==============================] - 12s 194us/step - loss: 0.0447 - acc: 0.9850 - val_loss: 0.0309 - val_acc: 0.9895
Epoch 6/20
60000/60000 [==============================] - 12s 194us/step - loss: 0.0412 - acc: 0.9868 - val_loss: 0.0338 - val_acc: 0.9893
Epoch 7/20
60000/60000 [==============================] - 12s 198us/step - loss: 0.0369 - acc: 0.9882 - val_loss: 0.